## Most Commonly used Optimization Algorithms

| Optimizer | Description | Use Case Example |
|---|---|---|
| SGD | Simple and effective but requires careful tuning of learning rate and momentum | Large-scale training with large datasets |
| Adam | Combines advantages of Adagrad and RMSprop; adaptive learning rates make it suitable for many tasks | Most deep learning tasks, including CNNs and RNNs |
| RMSprop | Adapts the learning rate for each parameter; effective for RNNs and problems where gradients vary significantly | RNNs and problems with varying gradients |

## Importing Required Libraries


In [65]:
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import to_categorical
from keras.src.optimizers.adam import Adam
from keras.optimizers import RMSprop
from keras.optimizers import SGD
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import classification_report
from google.colab import files
import warnings
warnings.filterwarnings('ignore')

### Reading Diabetes Dataset

In [36]:
uploaded = files.upload()
filename = next(iter(uploaded))

Saving winequality-red.csv to winequality-red (2).csv


### Create a DataFrame

In [37]:
wines_pd = pd.read_csv(filename)

In [38]:
wines_pd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1599 entries, 0 to 1598
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed acidity         1599 non-null   float64
 1   volatile acidity      1599 non-null   float64
 2   citric acid           1599 non-null   float64
 3   residual sugar        1599 non-null   float64
 4   chlorides             1599 non-null   float64
 5   free sulfur dioxide   1599 non-null   float64
 6   total sulfur dioxide  1599 non-null   float64
 7   density               1599 non-null   float64
 8   pH                    1599 non-null   float64
 9   sulphates             1599 non-null   float64
 10  alcohol               1599 non-null   float64
 11  quality               1599 non-null   int64  
dtypes: float64(11), int64(1)
memory usage: 150.0 KB


In [39]:
wines_pd

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5
1,7.8,0.880,0.00,2.6,0.098,25.0,67.0,0.99680,3.20,0.68,9.8,5
2,7.8,0.760,0.04,2.3,0.092,15.0,54.0,0.99700,3.26,0.65,9.8,5
3,11.2,0.280,0.56,1.9,0.075,17.0,60.0,0.99800,3.16,0.58,9.8,6
4,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5
...,...,...,...,...,...,...,...,...,...,...,...,...
1594,6.2,0.600,0.08,2.0,0.090,32.0,44.0,0.99490,3.45,0.58,10.5,5
1595,5.9,0.550,0.10,2.2,0.062,39.0,51.0,0.99512,3.52,0.76,11.2,6
1596,6.3,0.510,0.13,2.3,0.076,29.0,40.0,0.99574,3.42,0.75,11.0,6
1597,5.9,0.645,0.12,2.0,0.075,32.0,44.0,0.99547,3.57,0.71,10.2,5


In [40]:
wines_pd.isnull().sum()

,0
fixed acidity,0
volatile acidity,0
citric acid,0
residual sugar,0
chlorides,0
free sulfur dioxide,0
total sulfur dioxide,0
density,0
pH,0
sulphates,0


In [41]:
wines_pd.columns

Index(['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar',
       'chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density',
       'pH', 'sulphates', 'alcohol', 'quality'],
      dtype='object')

In [42]:
wines_pd.describe()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
count,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000
mean,8.319637,0.527821,0.270976,2.538806,0.087467,15.874922,46.467792,0.996747,3.311113,0.658149,10.422983,5.636023
std,1.741096,0.179060,0.194801,1.409928,0.047065,10.460157,32.895324,0.001887,0.154386,0.169507,1.065668,0.807569
min,4.600000,0.120000,0.000000,0.900000,0.012000,1.000000,6.000000,0.990070,2.740000,0.330000,8.400000,3.000000
25%,7.100000,0.390000,0.090000,1.900000,0.070000,7.000000,22.000000,0.995600,3.210000,0.550000,9.500000,5.000000
50%,7.900000,0.520000,0.260000,2.200000,0.079000,14.000000,38.000000,0.996750,3.310000,0.620000,10.200000,6.000000
75%,9.200000,0.640000,0.420000,2.600000,0.090000,21.000000,62.000000,0.997835,3.400000,0.730000,11.100000,6.000000
max,15.900000,1.580000,1.000000,15.500000,0.611000,72.000000,289.000000,1.003690,4.010000,2.000000,14.900000,8.000000


### Separate input features and target

In [43]:
X = wines_pd.drop('quality', axis=1)
y = wines_pd['quality']

In [44]:
X

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol
0,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4
1,7.8,0.880,0.00,2.6,0.098,25.0,67.0,0.99680,3.20,0.68,9.8
2,7.8,0.760,0.04,2.3,0.092,15.0,54.0,0.99700,3.26,0.65,9.8
3,11.2,0.280,0.56,1.9,0.075,17.0,60.0,0.99800,3.16,0.58,9.8
4,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4
...,...,...,...,...,...,...,...,...,...,...,...
1594,6.2,0.600,0.08,2.0,0.090,32.0,44.0,0.99490,3.45,0.58,10.5
1595,5.9,0.550,0.10,2.2,0.062,39.0,51.0,0.99512,3.52,0.76,11.2
1596,6.3,0.510,0.13,2.3,0.076,29.0,40.0,0.99574,3.42,0.75,11.0
1597,5.9,0.645,0.12,2.0,0.075,32.0,44.0,0.99547,3.57,0.71,10.2


In [45]:
y

,quality
0,5
1,5
2,5
3,6
4,5
...,...
1594,5
1595,6
1596,6
1597,5


### Preprocess Data (Scaling and Splitting)

In [46]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [47]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

### Function to Build and Train Model

In [48]:
def build_and_train_model(optimizer, name):
    model = Sequential()
    model.add(Dense(64, input_dim=X_train.shape[1], activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(1, activation='linear'))  # Linear for regression

    model.compile(loss='mean_squared_error', optimizer=optimizer, metrics=['mse'])

    print(f"\nTraining with {name} optimizer...")
    model.fit(X_train, y_train, epochs=100, batch_size=10, verbose=0)

    # Evaluate
    mse_train = model.evaluate(X_train, y_train, verbose=0)[0]
    mse_test = model.evaluate(X_test, y_test, verbose=0)[0]

    # Predict
    predictions = model.predict(X_test, verbose=0)
    rmse = np.sqrt(mean_squared_error(y_test, predictions))

    print(f"{name} - Train MSE: {mse_train:.2f}, Test MSE: {mse_test:.2f}, Test RMSE: {rmse:.2f}")

    return {
        'name': name,
        'mse_train': mse_train,
        'mse_test': mse_test,
        'rmse': rmse,
        'model': model
    }

### Train Models with Different Optimizers

In [49]:
results = []

# SGD
sgd = SGD(learning_rate=0.01, momentum=0.9)
results.append(build_and_train_model(sgd, "SGD"))

# Adam
adam = Adam(learning_rate=0.001)
results.append(build_and_train_model(adam, "Adam"))

# RMSprop
rmsprop = RMSprop(learning_rate=0.001)
results.append(build_and_train_model(rmsprop, "RMSprop"))


Training with SGD optimizer...
SGD - Train MSE: 0.10, Test MSE: 0.43, Test RMSE: 0.65

Training with Adam optimizer...
Adam - Train MSE: 0.18, Test MSE: 0.42, Test RMSE: 0.64

Training with RMSprop optimizer...
RMSprop - Train MSE: 0.18, Test MSE: 0.41, Test RMSE: 0.64


### Compare Results

In [50]:
results_df = pd.DataFrame(results)
print("\nComparison of Optimizers:")
print(results_df.sort_values(by='rmse'))


Comparison of Optimizers:
      name  mse_train  mse_test      rmse  \
2  RMSprop   0.178194  0.410810  0.640945   
1     Adam   0.181957  0.415129  0.644305   
0      SGD   0.099630  0.425139  0.652027   

                                        model  
2  <Sequential name=sequential_7, built=True>  
1  <Sequential name=sequential_6, built=True>  
0  <Sequential name=sequential_5, built=True>  


In [70]:
results_df

,name,accuracy,model
0,SGD,0.88750,"<Sequential name=sequential_8, built=True>"
1,Adam,0.86875,"<Sequential name=sequential_9, built=True>"
2,RMSprop,0.84375,"<Sequential name=sequential_10, built=True>"


### Access .summary() of Each Model

In [57]:
next(item for item in results if item['name'] == 'SGD')['model'].summary()

Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_14 (Dense)                │ (None, 64)             │           768 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_15 (Dense)                │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_16 (Dense)                │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 5,764 (22.52 KB)

 Trainable params: 2,881 (11.25 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2,883 (11.27 KB)

In [55]:
next(item for item in results if item['name'] == 'Adam')['model'].summary()

Model: "sequential_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_17 (Dense)                │ (None, 64)             │           768 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_18 (Dense)                │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_19 (Dense)                │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 8,645 (33.77 KB)

 Trainable params: 2,881 (11.25 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 5,764 (22.52 KB)

In [56]:
next(item for item in results if item['name'] == 'RMSprop')['model'].summary()

Model: "sequential_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_20 (Dense)                │ (None, 64)             │           768 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_21 (Dense)                │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_22 (Dense)                │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 5,764 (22.52 KB)

 Trainable params: 2,881 (11.25 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2,883 (11.27 KB)

**The core task is:** Develop an Artificial Neural Network (ANN) model to classify the quality of red wine based on its chemical properties. The dataset contains physicochemical measurements as input features and a wine quality score as the target. The objective is to train and evaluate a deep learning model that can predict whether a given wine sample falls into a particular quality category.

### Convert Quality Scores to Categories

In [58]:
def categorize_quality(quality):
    if quality <= 4:
        return 0  # Low
    elif quality <= 6:
        return 1  # Medium
    else:
        return 2  # High

In [59]:
wines_pd['quality_cat'] = wines_pd['quality'].apply(categorize_quality)

### Prepare input and output for classification

In [60]:
X = wines_pd.drop(['quality', 'quality_cat'], axis=1)
y = wines_pd['quality_cat']

### One-hot encode target

In [62]:
y_encoded = to_categorical(y)

### Train/Test Split

In [63]:
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

### Scale features

In [64]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

### Classification Model Function

In [66]:
def build_and_train_classifier(optimizer, name):
    model = Sequential()
    model.add(Dense(64, input_dim=X_train.shape[1], activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(3, activation='softmax'))  # 3 output classes

    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

    print(f"\nTraining with {name} optimizer...")
    model.fit(X_train, y_train, epochs=50, batch_size=16, verbose=0)

    # Evaluate model
    loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
    print(f"{name} - Accuracy: {accuracy*100:.2f}%")

    # Predictions
    y_pred_probs = model.predict(X_test, verbose=0)
    y_pred_classes = np.argmax(y_pred_probs, axis=1)
    y_true_classes = np.argmax(y_test, axis=1)

    print("\nClassification Report:")
    print(classification_report(y_true_classes, y_pred_classes, target_names=['Low', 'Medium', 'High']))

    return {
        'name': name,
        'accuracy': accuracy,
        'model': model
    }

### Train Models with Classifiers

In [67]:
results = []

# SGD
sgd = SGD(learning_rate=0.01, momentum=0.9)
results.append(build_and_train_classifier(sgd, "SGD"))

# Adam
adam = Adam(learning_rate=0.001)
results.append(build_and_train_classifier(adam, "Adam"))

# RMSprop
rmsprop = RMSprop(learning_rate=0.001)
results.append(build_and_train_classifier(rmsprop, "RMSprop"))


Training with SGD optimizer...
SGD - Accuracy: 88.75%

Classification Report:
              precision    recall  f1-score   support

         Low       0.33      0.09      0.14        11
      Medium       0.90      0.97      0.93       262
        High       0.83      0.62      0.71        47

    accuracy                           0.89       320
   macro avg       0.69      0.56      0.59       320
weighted avg       0.87      0.89      0.87       320


Training with Adam optimizer...
Adam - Accuracy: 86.87%

Classification Report:
              precision    recall  f1-score   support

         Low       0.50      0.09      0.15        11
      Medium       0.89      0.95      0.92       262
        High       0.71      0.57      0.64        47

    accuracy                           0.87       320
   macro avg       0.70      0.54      0.57       320
weighted avg       0.85      0.87      0.85       320


Training with RMSprop optimizer...
RMSprop - Accuracy: 84.38%

Classification

### Compare Model Accuracies

In [68]:
results_df = pd.DataFrame(results)
print("\nAccuracy Comparison of Optimizers:")
print(results_df[['name', 'accuracy']].sort_values(by='accuracy', ascending=False))


Accuracy Comparison of Optimizers:
      name  accuracy
0      SGD   0.88750
1     Adam   0.86875
2  RMSprop   0.84375


### View Model Architectures

In [69]:
for item in results:
    print(f"\nModel Summary - {item['name']}")
    item['model'].summary()


Model Summary - SGD


Model: "sequential_8"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_23 (Dense)                │ (None, 64)             │           768 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_24 (Dense)                │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_25 (Dense)                │ (None, 3)              │            99 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 5,896 (23.04 KB)

 Trainable params: 2,947 (11.51 KB)

...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_29 (Dense)                │ (None, 64)             │           768 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_30 (Dense)                │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_31 (Dense)                │ (None, 3)              │            99 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 5,896 (23.04 KB)

 Trainable params: 2,947 (11.51 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2,949 (11.52 KB)